In [97]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day15input.txt")

# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:


In [98]:
test_data = ["Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3"]

In [99]:
def parse_data(data):
    returnData = []
    for d in data:
        pair = d.split(":")
        sensor = pair[0].split(",")
        d1 = eval(sensor[0].removeprefix("Sensor at x="))
        d2 = eval(sensor[1].removeprefix(" y="))
        beacon = pair[1].split(",")
        d3 = eval(beacon[0].removeprefix(" closest beacon is at x="))
        d4 = eval(beacon[1].removeprefix(" y="))
        manhattan = abs(d3-d1) + abs(d4-d2)
        upper = d2+manhattan
        lower = d2-manhattan
        returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        #print(d1, d2, d3, d4)
    return returnData

parsed_test_data = parse_data(test_data)
parsed_elf_data = parse_data(elf_data)

import sys


So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

                1    1    2    2
        0    5    0    5    0    5
    0 ....S.......................
    1 ......................S.....
    2 ...............S............
    3 ................SB..........
    4 ............................
    5 ............................
    6 ............................
    7 ..........S.......S.........
    8 ............................
    9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....


In [100]:
def get_grid_dim(pd):
    min_x = pd[0][0]
    max_x = pd[0][0]
    # min_y = pd[0][1]
    # max_y = pd[0][1]
    for j in pd:
        # print(i)
        # print(j)
        # print(j[0])
        # print(j[1])
        # min_y = min(j[1], j[3], min_y)
        # max_y = max(j[1], j[3], max_y)
        min_x = min(j[0], j[2], min_x)
        max_x = max(j[0], j[2], max_x)

    return (min_x, max_x)


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

                1    1    2    2
        0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
    0 ....S...#####...............
    1 .......#######........S.....
    2 ......#########S............
    3 .....###########SB..........
    4 ....#############...........
    5 ...###############..........
    6 ..#################.........
    7 .#########S#######S#........
    8 ..#################.........
    9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).


In [101]:
def setup_row_from_scratch(min_x, max_x, row_num, parsed_data):
    new_line = []
    for x in range(min_x, max_x+1):    
        new_line.append(".")
    s = len(parsed_data)
    s0 = 0
    # print("Empty row set up, checking ", s, " sensors")
    for sensor in parsed_data:
        sensor_x = sensor[0]-min_x
        sensor_y = sensor[1]  # no need to normalise to 0 index
        beacon_x = sensor[2]-min_x
        beacon_y = sensor[3]
        s0 = s0+1
        # print("Checking sensor ", s0, " of ", s)
        if sensor_y == row_num:
            new_line[sensor_x] = "S"
            # print("sensor added at ", sensor_x, row_num)
        elif beacon_y == row_num:
            new_line[beacon_x] =  "B"
            # print("beacon added at ", beacon_x, row_num)
    # print("Finished, returning row")
    return new_line


def check_row_from_scratch(row, row_no, min_x, max_x, parsed_data):
    for sensor in parsed_data:
        # returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        if row_no in range(sensor[6], sensor[5]): #affected
            sensor_x = sensor[0]-min_x
            sensor_y = sensor[1]
            manhattan = sensor[4]
            beacon_x = sensor[2]-min_x
            beacon_y = sensor[3]
            
            diff = abs(sensor_y - row_no) 
            x_range = manhattan - diff 
            x_left = sensor_x - x_range
            # if x_left<0:
            #     print("XLEFT = ",x_left)    
            x_left = max(x_left, 0)
            x_right = sensor_x + x_range
            # if x_right>len(row)-1:
            #     print("XRIGHT = ",x_left)   
            x_right = min(x_right, (len(row)-1))

            #print("Checking sensor at ", sensor_x, sensor_y, " over range ", x_left, x_right, " with manhattan ", manhattan, " with diff ", diff, " x_range ", x_range, " and min_x is ", min_x)
            for x in range(x_left, x_right+1):
                if row[x]==".":
                    row[x] = "#"
                    #print(x, "#")
                # else: 
                    #print(x, row[x])
    na = row.count("#") + row.count("S")
    if row.__contains__("."):
        poss = row.index(".")
    else: 
        poss = -1
    #print(row)
    return (na, poss, min_x, max_x)


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                    1    1    2    2
        0    5    0    5    0    5
    9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.


In [102]:
def explore_row(row_num, parsed_data, part, is_test):
    (min_x, max_x) = get_grid_dim(parsed_data)
    if part == 1:
        min_x = min(min_x, -3000000)
        max_x = max(max_x, 8000000)
    elif is_test: 
        min_x = 0
        max_x = 20
    else: 
        min_x = 0
        max_x = 4000000
    # print("get grid_dim ", min_x, max_x, " now starting setup row ", row_num)
    row = setup_row_from_scratch(min_x, max_x, row_num, parsed_data)
    # print("Row set up length ", len(row), " starting to check row ", row_num)
    return check_row_from_scratch(row, row_num, min_x, max_x, parsed_data)
    

(notavailable, possible, min_x, max_x) = explore_row(10, parsed_test_data, 1, True)
print(notavailable)


26



In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [103]:
#(notavailable2, possible2, min_x2, max_x2) = explore_row(2000000, parsed_elf_data, 1, False)
#print(notavailable2)

# --- Part Two ---

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?



In [104]:
def calc_tuning_frequency(x, y):
    return ((x*4000000)+y)

In [105]:
def look_for_tuning_freq(parsed_data, part, is_test):
    y = 0
    found = False
    limit = 20
    if part==2:
        if not is_test:
            limit = 4000000
    while y<limit and found is False:
        (notavailable3, possible3, min_x3, max_x3) = explore_row(y, parsed_data, part, is_test)
        print(y, notavailable3, possible3, min_x3, max_x3)
        if possible3 > -1 and possible3-min_x3 <= max_x3:# and possible3<notavailable3:
            found = True
        else: 
            y = y+1
    print(notavailable3, possible3, y, min_x3, max_x3)
    return calc_tuning_frequency(possible3,y)



In [106]:
print(look_for_tuning_freq(parsed_test_data, 2, True))

#beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

0 21 -1 0 20
1 21 -1 0 20
2 21 -1 0 20
3 20 -1 0 20
4 21 -1 0 20
5 21 -1 0 20
6 21 -1 0 20
7 21 -1 0 20
8 21 -1 0 20
9 21 -1 0 20
10 20 -1 0 20
11 20 14 0 20
20 14 11 0 20
56000011


In [ ]:
#     print(na, possible3, y, min_x3, max_x3)


print(look_for_tuning_freq(parsed_elf_data, 2, False))


*108000000 is too low*